In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras as krs
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
from sklearn import preprocessing
import sklearn

In [ ]:
type_Ann = 1 # 0 = all_soils, 1 = fine_soils, 2 = granular_soils


##### Do not change any part of the code below

nameFile = 'input.xlsx'

df = pd.read_excel(nameFile)

if type_Ann == 2:
    dfRNA = df[['D50','w','gd']]
else:
    dfRNA = df[['D50',"PF", "wL", "PI", 'w','gd']]

dfRNA["gd"] = (dfRNA["gd"].values-13.27)/3.48

inputs = dfRNA.values

def f(x):
    max_value = (2.63-0.8752)/1.067693884
    min_value = (-2.21-0.8752)/1.067693884
    a = (x+1)/2
    b = a*(max_value-min_value)
    return b+min_value

name_File = ["", "_Fines", "_Granular"]

dic_output = {}

for i in np.arange(0, 20, 1):
    model = krs.models.load_model(f"1. tc/Models{name_File[type_Ann]}/Modelo{i}")
    print(f"Modelo_{i}")
    dic_output[f"Modelo_{i}"] = model.predict(inputs).reshape(1,-1)[0]


df_output = pd.DataFrame(dic_output)

name_File = ["All", "Fine", "Granular"]

model = CatBoostRegressor().load_model(f"1. tc/CatBoost_tc{name_File[type_Ann]}")

output = np.exp(f(model.predict(df_output.values))*1.067693884+0.847280899)

dic_output = pd.DataFrame({"tc":output})

dic_output.to_excel("output.xlsx")